# 7. Capital Protection

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate

from scipy.optimize import fsolve
from scipy.stats import norm
from call_put_bs import call_put_bs
from gbm_d import gbm_d
from hpr_cp import hpr_cp
from yf import yf
from find_beta import find_beta

## 7.1 Payoff of an uncapped capital protection product

The payoff of an uncapped capital protection product written on one underlying is

\begin{equation*}
g(S_T)=kN+\frac{zN}{S_0}\max\{S_T-X,0\}
\end{equation*}

where $k$ is the capital protection and $z$ is the participation (both in multiples of the denomination $N$). In the product described below, we have $k=1.01$ and $z=1$.

<img src='Image/CP_ASIAN_RAIFF.jpg' alt='Drawing' style='width: 600px;'/>

## 7.2. Capital protection products on multiple underlyings

We exemplarily consider the capital protection product written on multiple underlyings described in figure below. 

<img src='Image/MCP_LEON_1.jpg' alt='Drawing' style='width: 600px;'/>
<img src='Image/MCP_LEON_2.jpg' alt='Drawing' style='width: 600px;'/> 

To find this expectation we again model the time evolution of the stock prices ${\bf S}_t$ as a $d$-dimensional geometric Brownian motion; for this model it seems that this expectation can not be calculated analytically and one has to rely on numerical methods.

### Example 1.

We consider the product written on Nestle $\rightarrow 1$, Novartis $\rightarrow 2$, Roche $\rightarrow 3$, Swiss Re $\rightarrow 4$ and Zurich $\rightarrow 5$ described in the above term sheet and check the issue price $V_{{\rm cp},5}= 1000$ CHF.

The initial stock prices $\textbf{S}_0=(S_0^1,\ldots,S_0^5)$ are from the term sheet; to estimate the dividend yields $q_i=\ln(1+D_i/S_0^i)$ we take the dividends payments made in $2019$, thus $(D_1,D_2,D_3,D_4,D_5)=(2.45,2.89,8.7,5.2,19.33)$. The risk free is taken from the term sheet ("Bondfloor at issuance") as $79.52e^{rT}=80$ $\Rightarrow$ $r=\frac{1}{T}\ln(80/79.52)$. We take the implied volatilities 

\begin{equation*}
\boldsymbol{\sigma}^{\textrm{i}}=\textrm{diag}(0.1703,0.1817,0.1765,0.1901,0.1788)\;.
\end{equation*}

In [3]:
T = yf((27,6,2019),[(27,6,2029)])
s0 = [100.56,89.21,273.55,98.48,339.4]; D = [2.45,2.89,8.7,5.2,19.33]; r = np.log(80/79.52)/T
q = np.log(1+np.reshape(np.asarray(D),[5,1])/np.reshape(np.asarray(s0),[5,1]))
sigma = np.diag([0.1703,0.1817,0.1765,0.1901,0.1788]); lam = 0.878; 
rho = (1-lam)*np.eye(5)+lam*np.ones((5,5)); Sigma = sigma.dot(rho).dot(sigma)
N = 1000; c = np.inf; k = 0.8; z = 4.59; gamma = 0.8;
mu = r-q;

In [4]:
# input parameters (Monte Carlo simulation)
n = 10**5; v = np.zeros(n)

In [5]:
# Monte Carlo simulation
for j in range(0,n):
    S = gbm_d(s0,mu,Sigma,[],T,T)[0] 
    v[j] = np.maximum(np.minimum(1+c-k,z*(np.min(S[-1,:]/S[0,:])-gamma)),0) # payoff worst-of-option

Vcwo = np.exp(-r*T)*np.mean(v)
V = N*(np.exp(-r*T)*k+Vcwo); display((V[0],Vcwo[0]))

(979.6421537049524, 0.18444215370495257)

## 7.3. Capital Protection Product With Asian Option Component

As an example, we consider the capital protection product in the figure below.

<img src='Image/CP_ASIAN_RAIFF.jpg' alt='Drawing' style='width: 500px;'/>

with $A_T$ the (discrete, arithmetic) average. The expression $\max\{A_T-S_0,0\}$ is the payoff of a fixed strike asian call with strike $X=S_0$. The time $t$-value of this capital protection product is thus given by

\begin{equation*}
V_{\rm cp}(s,t)=e^{-r(T-t)}kN+\frac{zN}{S_0}V_{{\rm fs},c}(s,t)\;,
\end{equation*}

where we denote by $V_{{\rm fs},c}(s,t)$ the value of a fixed strike asian call. Even for the simple model of Black and Scholes (geometric Brownian motion) there is no formula to calculate $V_{{\rm fs},c}(s,t)$ and (once more) one needs numerical methods. For the parameters given in the term sheet and the market values $\sigma=22.2\%$, $r=\ln(1+0.0308)$, $q=\ln(1+0.0207)$ (values from Bloomberg) and $s=S_0=2792.62$ (ATX at $16/5/2011$) a Monte Carlo simulation based on $10^6$ paths yields the issue price $V_{\rm cp}(s,0)\approx 974.84$; in particular $V_{{\rm fs},c}(s,0) \approx 374.91$. Notice the rather large agio; according to the term sheet, the investor has to pay Eur $1030$.

In [ ]:
# input parameters (model, contract)
s0 = [2792.62]; q = [0.0205]; r = np.log(1+0.0308)
Sigma = [[0.222**2]]
X = s0; N = 1000; k = 1; z = 1.05
T = yf((16,5,2011),[(12,5,2017)])[0]
tj = yf((16,5,2011),[(14,5,2012),(13,5,2013),(12,5,2014),(12,5,2015),(12,5,2016),(12,5,2017)]).tolist()
mu = [r-x for x in q]; q = np.asarray(q)

In [ ]:
# input parameters (Monte Carlo simulation)
n = 10**3; v = np.zeros(n)

In [ ]:
# Monte Carlo simulation
for j in range(0,n):
    D = gbm_d(s0,mu,Sigma,tj,T,T) 
    S = D[0][1:] # simulated stock prices, excluding s0
    AT = np.mean(S) # average price at maturity
    
    v[j] = np.maximum(AT-X,0) # payoff

Vfsc = np.exp(-r*T)*np.mean(v)
Vcp = N*(np.exp(-r*T)*k+z/np.asarray(s0)*Vfsc); display((Vcp[0],Vfsc))

Observe that the value $V_{{\rm fs},c}(s,0)\approx 374.91$ is (much) lower than the price $V_c(s,0)\doteq 587.40$ of the corresponding european call. Whence, by considering the average price of the underlying at maturity rather than its final value gives the issuer the possibility to offer a higher capital protection and/or participation rate to the client.

This means that once the density $f_{R^c}$ of the log-returns of the underlying is known, we can explicitly calculate the density of the holding period return of the capital protection products.

In the figure below we plot - using the Python function <span style="color:orange">hpr_cp.py</span> - the pdfs defined above for the products defined in the script in the case that the log-returns of the underlying $R^c_{t,T}\sim\mathcal{N}(\mu_S,\sigma_S^2)$ are normal. The parameter values are $t=0$, $T=3$, $S_0=640$, $v_0=1.03$, $X=X_1=S_0$, $X_2=1.25S_0$, $k=0.95$, and $z=0.8$ for the uncapped product, $z=1.3907$, $c=-0.05+0.25z$ for the capped product. Furthermore, the parameters of the geometric Brownian motion modelling $S_T$ are $\mu=\ln(1.03)$ and $\sigma=0.19$. We compare these with the density $f_{R_{\rm u}}$ of the return $R_{{\rm u},t,T}=\frac{S_T}{S_t}-1$ of a direct investment into the underlying.

In [ ]:
# the uncapped product
hpr_cp(St=640,S0=640,vt=1.03,k=0.95,z=0.8,X1=640,X2=[],c=0,T=3,mu=np.log(1.03),sigma=0.19);

# the capped product
hpr_cp(St=640,S0=640,vt=1.03,k=0.95,z=1.3907,X1=640,X2=1.25*640,c=-0.05+0.25*1.3907,T=3,mu=np.log(1.03),sigma=0.19);

Likewise, we have for the capped product

\begin{eqnarray*}
\mu_{\rm ccp} & = & \mathbb{E}[R_{{\rm ccp},t,T}]=\int_\alpha^\beta xf_{R_{\rm ccp}}(x){\rm d}x+p_1\alpha+p_2\beta\\
\sigma^2_{R_{\rm ccp}} & = & \mathbb{E}[(R_{{\rm ccp},t,T}-\mu_{\rm ccp} )^2] \\
& = & \int_{\alpha}^\beta x^2f_{R_{\rm ccp}}(x){\rm d}x+\alpha^2p_1+\beta^2p_2-\mu_{\rm ccp}^2\;.
\end{eqnarray*}

In case that the log-returns of the underlying are normal, all the above integrals can be calculated analytically. However, for more realistic distributions, these integrals can only be evaluated numerically. In Python this can be accomplished using <span style="color:orange">quad</span>. We use the self-written Python function <span style="color:orange">hpr_cp.py</span> to calculate the expected return and the volatility of (capped or uncapped) capital protection products. As an example, consider the uncapped capital protection product above. Using <span style="color:orange">hpr_cp.py</span> we then obtain $\mathbb{E}[R_{{\rm cp},t,T}]\doteq 0.0684$ and $\sigma_{R_{\rm cp}}\doteq 0.2213$.

In [ ]:
# the uncapped product
hpr_cp(St=640,S0=640,vt=1.03,k=0.95,z=0.8,X1=640,X2=[],c=0,T=3,mu=np.log(1.03),sigma=0.19)

In [ ]:
Likewise, for the capped product from above

In [ ]:
# the capped product
hpr_cp(St=640,S0=640,vt=1.03,k=0.95,z=1.3907,X1=640,X2=1.25*640,c=-0.05+0.25*1.3907,T=3,mu=np.log(1.03),sigma=0.19)